In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

In [ ]:
# 读取四张表的数据
#读取了订单和产品id的关联，csv比较大
prior = pd.read_csv("./data/instacart/order_products__prior.csv")

In [ ]:
#产品id，与过道的对应
products = pd.read_csv("./data/instacart/products.csv")

In [ ]:
#订单id和用户id的对应，csv比较大
orders = pd.read_csv("./data/instacart/orders.csv")

In [ ]:
#超市的过道，过道放的产品的品类
aisles = pd.read_csv("./data/instacart/aisles.csv")

In [ ]:
prior.head()  #订单id，产品id，下面只需要掌握订单和产品的关联即可

In [ ]:
products.head() #产品id，产品名称，过道id

In [ ]:
orders.head() #订单id，用户id

In [ ]:
aisles.head() #过道id，对应过道里放了哪些产品类别

In [ ]:
# 合并四张表到一张表  （用户-物品类别），在这里挂了，用下阿里云
#订单表和产品表关联
#用户买了哪些订单，订单里面有哪些产品，产品对应的过道，过道对应的产品类别
_mg = pd.merge(prior, products, on=['product_id', 'product_id'])
_mg = pd.merge(_mg, orders, on=['order_id', 'order_id'])
mt = pd.merge(_mg, aisles, on=['aisle_id', 'aisle_id'])

In [ ]:
mt.head(10)

In [ ]:
mt.shape

In [ ]:
mt.isnull().sum()/mt.shape[0]  #数据缺失率，没有缺失值

In [ ]:
# 交叉表（特殊的分组工具），如果在这一步内存不够了，重启一下jupyter，还是不行，筛选一部分用户的数据，进行聚类
cross = pd.crosstab(mt['user_id'], mt['aisle'])

In [ ]:
cross.head(10)

In [ ]:
#针对20万用户，有134个特征
cross.shape

In [ ]:
type(cross)

In [ ]:
# 进行主成分分析,那么大家还记得不？，保留90%的成分，实际特征数目减少了几倍，134--》27
pca = PCA(n_components=0.9)

In [ ]:
data = pca.fit_transform(cross)  #还记得fit_transform的作用不？
data.shape

In [ ]:
type(data)

In [ ]:
# 把样本数量减少,为了下面画图方便，这里只取500个样本
x = data[:500]
x.shape

In [ ]:
x[0:10]

In [ ]:
x.max()

In [ ]:
x.min()

In [ ]:
# 假设用户一共分为四个类别
#n_clusters是分为几个类别，是主要的超参
km = KMeans(n_clusters=4)

In [ ]:
km.fit(x)  #开始训练

In [ ]:
#开始预测
predict = km.predict(x)

In [ ]:
print(predict)  #预测得到的是每一个样本的类别

In [ ]:
import numpy as np
np.unique(predict)

In [ ]:
# 显示聚类的结果
plt.figure(figsize=(20, 20))

In [ ]:
# 建立四个颜色的列表
colored = ['orange', 'green', 'blue', 'purple']
colr = [colored[i] for i in predict] #根据类别给每个样本赋予颜色
#去选择任意两个特征来看聚类的图，可以自行调整,第2个，第20个特征
plt.scatter(x[:, 1], x[:, 19], color=colr)

In [ ]:
# 评判聚类效果，轮廓系数，聚类效果很难超过0.7，可以通过这个得分去调整上面KMeans的n_clusters
silhouette_score(x, predict)

In [ ]:
#改为2时的结果
km = KMeans(n_clusters=3)
km.fit(x)  #开始训练
predict = km.predict(x)
print(silhouette_score(x, predict))
predict

In [ ]:
# 通过分类后数据，去看原始样本的特点
cross.iloc[0]

In [ ]:
cross.iloc[0]
predict[26]
cross.iloc[26]

In [ ]:
#改为3时的结果
km = KMeans(n_clusters=3)
km.fit(x)  #开始训练
predict = km.predict(x)
silhouette_score(x, predict)

# 四种异常值检测的方法

## 1.基于分位数找异常值

In [ ]:
np.random.seed(42)  # 为了结果的可重复性设置随机种子
X = np.random.rand(50, 1)  # 2维数据
X

In [ ]:
X[0,0]=3
X[1,0]=2

In [ ]:
import pandas as pd

# 假设df是一个Pandas DataFrame，包含了你想要分析的数据集
# 'column_name'是你想要检测异常值的列名

#随机一个dataframe
df=pd.DataFrame(X,columns=['column_name'])

# 设置分位数，例如四分位数
Q1 = df['column_name'].quantile(0.25)  # 第一四分位数（25%）
Q3 = df['column_name'].quantile(0.75)  # 第三四分位数（75%）

# 计算四分位数范围（IQR）
IQR = Q3 - Q1

# 定义下界和上界来识别异常值
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 标记异常值
df['outlier'] = df['column_name'].apply(lambda x: 'Yes' if x < lower_bound or x > upper_bound else 'No')

# 打印出被标记为异常值的行
df[df['outlier'] == 'Yes']